In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl
from IPython.display import display, Markdown

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

# Market Data Subscriptions

In [ ]:
start_time = datetime(2025, 3, 5)

# ticker subscriptions
symbols = ["BTC/USD:CXTALP", "SPX", "NVDA", "SPY", "QQQ", "/ESH25:XCME"]
# symbols = ["/ESH25:XCME"]
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX", "/ESH25:XCME"]:
# for symbol in ["/ESH25:XCME"]:
    for interval in ["1d", "1h", "30m", "15m", "5m", "m"]:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

In [4]:
# forward fill
for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:
# for symbol in ["/ESH25:XCME"]:
    for interval in ["1d", "1h", "30m", "15m", "5m", "1m"]:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds

In [ ]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

In [ ]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

In [7]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    HorizontalLine,
    VerticalLine,
)
from tastytrade.analytics.indicators.momentum import macd
from tastytrade.messaging.models.events import CandleEvent
import re

from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription
from tastytrade.config import RedisConfigManager
from tastytrade.analytics.visualizations.utils import get_opening_range

import pytz

month = 3
day = 6

subscription = RedisSubscription(config=RedisConfigManager())
await subscription.connect()

et_tz = pytz.timezone("America/New_York")
market_open = datetime(2025, month, day, 9, 30, tzinfo=et_tz)
morning_end = datetime(2025, month, day, 11, 30, tzinfo=et_tz)
lunch_end = datetime(2025, month, day, 13, 30, tzinfo=et_tz)
market_close = datetime(2025, month, day, 16, 0, tzinfo=et_tz)

streamer = MarketDataProvider(subscription, influxdb)

start = datetime(2025, month, day, 9) + timedelta(hours=5)
stop = datetime(2025, month, day, 17) + timedelta(hours=5)

In [9]:
candle_symbol = "SPX{=m}"
candle_symbol = "/ESH25:XCME{=m}"

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=re.sub(r"\{=.*?\}", "{=d}", candle_symbol),
            start=start.date() + timedelta(days=-1),
            stop=start.date(),
            debug_mode=True,
        )
        .to_dicts()
        .pop()
    )
)

In [ ]:
or5 = await get_opening_range(
    streamer,
    "SPX{=m}",
    5,
    date=start.date(),
)

or15 = await get_opening_range(
    streamer,
    "SPX{=m}",
    15,
    date=start.date(),
)

or30 = await get_opening_range(
    streamer,
    "SPX{=m}",
    30,
    date=start.date(),
)

In [11]:
levels = [
    HorizontalLine(
        price=prior_day.close,
        color="#FF66FE",  # Orange
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.high,
        color="#4CAF50",  # Green
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.low,
        color="#F44336",  # Red
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=or5.high,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or5.low,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or15.high,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.high != or5.high else 0.0,
    ),
    HorizontalLine(
        price=or15.low,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.low != or5.low else 0.0,
    ),
    HorizontalLine(
        price=or30.high,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.high != or15.high else 0.0,
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
    ),
]

In [12]:
executions = [
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=1, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=4, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        time=datetime(2025, 2, 27, 14, 30) + timedelta(minutes=20),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=1, minutes=40),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=4, minutes=0),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        time=datetime(2025, 3, 3, 14, 30) + timedelta(hours=0, minutes=20),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=1, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Pass",
    ),
    VerticalLine(
        time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=2, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=0, minutes=45),
        color="#555555",
        line_dash="dot",
        label="Pass",
    ),
    VerticalLine(
        time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=2, minutes=10),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
]

In [ ]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd = macd(candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9)

plot_macd_with_hull(
    df_macd,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol.replace("m", "5m"),
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd_5m = macd(
    candles_5m, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
# debug
df_macd_5m.to_pandas()

In [ ]:
await streamer.subscribe("CandleEvent", "SPX{=*}")
await asyncio.sleep(2)
streamer.frames.keys()

In [ ]:
streamer["CandleEvent:SPX{=m}"].sort("time", descending=True)

In [ ]:
await dxlink.close()